In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('data.csv')

In [15]:
df

,Unnamed: 0,p_cntrl4224,p_cncr8407,p_cntrl6764,p_cncr3273,p_cntrl1262,p_cncr8861,p_cncr5984,p_cncr8964,p_cncr7142,...,p_cntrl3871,p_cntrl9527,p_cncr8117,p_cntrl3797,p_cncr2900,p_cncr4395,p_cncr2896,p_cncr3688,p_cncr3488,p_cncr5857
0,10100011111111100001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1001000100110001111,16,22238,2429,205,211477,235,4040,89,3297,...,810,569497,90,2639,5750326,1664372,57,12861,2803,5306
2,11100110010111011110,200306,1883652,75266,137451,33655,19243,210799,75,21905373,...,29868,16047,102685,2136875,372,20168,9549,907939,2165,124579
3,1001000101000110111,3589,5246,44866,4296,448249,390585,3171,13306829,410,...,55173,350512,67439,23863,24240,849656,480214,13620,2346351,210529
4,10101011001001001011,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4984,100111010101,1084,92,8425,5749,173909,686,9872,184,3017,...,216,369164,3271,9210,1580975,4131,4592,126,8771,4882
4985,1000100001011011010,154963,119032,114491,34791,30286,210002,119774,3188,41409,...,124886,157290,1183759,498661,1303,1415342,561366,73925,4206,113270
4986,11111000011101001001,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4987,1101000101101111110,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [18]:
DNAs = df['Unnamed: 0'].values

len(str(DNAs[4987]))

19